In [1]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

## Graduation Rates

In [2]:
grad_rates_1011 = pd.read_excel("data/grad_rates/Grad_Rates_2010-11_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1112 = pd.read_excel("data/grad_rates/Grad_Rates_2011-12_cohort.xls", sheet_name=1, usecols="B:H", skiprows=5)
grad_rates_1213 = pd.read_excel("data/grad_rates/Grad_Rates_2012-13_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1314 = pd.read_excel("data/grad_rates/Grad_Rates_2013-14_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1011.head(3)

,School ID,School Name,Cohort,Subgroup,Students,Grads,Rate
0,31,Reti-Wrap Transition Center,2010-2011,All Students,28.0,5,0.178571
1,32,Family Support Center,2010-2011,All Students,3.0,s,s
2,101,John Bartram High School,2010-2011,All Students,281.0,203,0.72242


In [3]:
grad_rates_1112 = grad_rates_1112.rename(columns={'Subgrouop': 'Subgroup', '9th Grade Cohort': 'Cohort'})
grad_rates_1112 = grad_rates_1112[grad_rates_1112.Subgroup == 'All Students']

In [4]:
grad_rates = pd.concat([grad_rates_1011, grad_rates_1112, grad_rates_1213, grad_rates_1314]).reset_index(drop=True)
grad_rates = grad_rates.rename(columns={'Rate': 'GradRate'})
print(grad_rates.shape)
grad_rates.head()

(368, 7)


,School ID,School Name,Cohort,Subgroup,Students,Grads,GradRate
0,31,Reti-Wrap Transition Center,2010-2011,All Students,28,5,0.178571
1,32,Family Support Center,2010-2011,All Students,3,s,s
2,101,John Bartram High School,2010-2011,All Students,281,203,0.72242
3,102,West Philadelphia High School,2010-2011,All Students,182,123,0.675824
4,103,High School Of The Future,2010-2011,All Students,146,104,0.712329


In [5]:
# replace 's' with NaN
# 's' means supressed
grad_rates = grad_rates.replace('s', np.nan)

# if grad count is suprressed then suppress student count too
grad_rates['Students'] = grad_rates['Students'].where(~pd.isnull(grad_rates['Grads']), np.nan)

# convert School ID to string
grad_rates['School ID'] = grad_rates['School ID'].astype(str)

# all subgroups are the same (equals 'All Students')
grad_rates.drop('Subgroup', axis=1, inplace=True)

# remove school names. They'll be added later
grad_rates.drop('School Name', axis=1, inplace=True)

In [6]:
grad_rates.head()

,School ID,Cohort,Students,Grads,GradRate
0,31,2010-2011,28.0,5.0,0.178571
1,32,2010-2011,NaN,NaN,NaN
2,101,2010-2011,281.0,203.0,0.722420
3,102,2010-2011,182.0,123.0,0.675824
4,103,2010-2011,146.0,104.0,0.712329


In [7]:
# convert rows to columns
# get one row per school
grad_rates.set_index(['School ID', 'Cohort'], inplace=True)
grad_rates = grad_rates.unstack().reset_index()
grad_rates.columns = [' '.join(col).strip() for col in grad_rates.columns.values]
grad_rates.head()

,School ID,Students 2010-2011,Students 2011-2012,Students 2012-2013,Students 2013-2014,Grads 2010-2011,Grads 2011-2012,Grads 2012-2013,Grads 2013-2014,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014
0,101,281.0,293.0,239.0,217.0,203.0,177.0,130.0,126.0,0.722420,0.604096,0.543933,0.580645
1,102,182.0,171.0,156.0,118.0,123.0,107.0,83.0,76.0,0.675824,0.625731,0.532051,0.644068
2,103,146.0,159.0,101.0,124.0,104.0,127.0,84.0,91.0,0.712329,0.798742,0.831683,0.733871
3,105,68.0,67.0,73.0,76.0,62.0,65.0,69.0,73.0,0.911765,0.970149,0.945205,0.960526
4,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# df.columns = [' '.join(col).strip() for col in df.columns.values]
# df.head()

In [9]:
# get 4-year combined graduation rate
grad_rates['Students Total'] = grad_rates[['Students 2010-2011', 'Students 2011-2012', \
                                          'Students 2012-2013','Students 2013-2014']].sum(axis=1)
            
grad_rates['Grads Total'] = grad_rates[['Grads 2010-2011', 'Grads 2011-2012', \
                                        'Grads 2012-2013','Grads 2013-2014']].sum(axis=1)
            
grad_rates['GradRate Total'] = grad_rates['Grads Total'] / grad_rates['Students Total']
            
grad_rates.head()

,School ID,Students 2010-2011,Students 2011-2012,Students 2012-2013,Students 2013-2014,Grads 2010-2011,Grads 2011-2012,Grads 2012-2013,Grads 2013-2014,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total
0,101,281.0,293.0,239.0,217.0,203.0,177.0,130.0,126.0,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476
1,102,182.0,171.0,156.0,118.0,123.0,107.0,83.0,76.0,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415
2,103,146.0,159.0,101.0,124.0,104.0,127.0,84.0,91.0,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038
3,105,68.0,67.0,73.0,76.0,62.0,65.0,69.0,73.0,0.911765,0.970149,0.945205,0.960526,284.0,269.0,0.947183
4,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [10]:
# subset columns
df = grad_rates[['School ID', 'GradRate 2010-2011', 'GradRate 2011-2012', 'GradRate 2012-2013', \
                 'GradRate 2013-2014', 'Students Total', 'Grads Total', 'GradRate Total']]

## Teacher Attendance Data

In [11]:
# teacher attendance
teach_attnd = pd.read_excel("data/Teacher_Attend_1516.xlsx", sheet_name=0)
# teach_attnd.set_index(['School ID'], inplace=True)

# convert School ID to string
teach_attnd['School ID'] = teach_attnd['School ID'].astype(str)

df2 = pd.merge(df, teach_attnd, how='outer', on='School ID')
df2.drop(['School Year', 'ULCS', 'SDP Avg'], axis=1, inplace=True)
df2.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %
0,101,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476,93.5
1,102,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415,94.0
2,103,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038,91.4
3,105,0.911765,0.970149,0.945205,0.960526,284.0,269.0,0.947183,97.0
4,106,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


## School Progress Report (SPR) Data

In [12]:
# read SPR data
spr = pd.read_excel('data/SPR_SY1617_Data/SPR_SY1617_School_Metric_Scores_20180206.xlsx', sheet_name=1)

# convert School ID to string
spr = spr.rename(columns={'SRC School ID': 'School ID'})
spr['School ID'] = spr['School ID'].astype(str)
# spr.set_index(['School ID', 'Report'], inplace=True)

spr['Full Address'] = spr['Street Address'] + ', ' + spr['City'] + ', ' + spr['State'] + ' ' + spr['Zip Code'].astype(str)
spr.drop(['Street Address', 'City', 'State', 'Zip Code', 'Phone Number', 'Fax Number'], axis=1, inplace=True)
spr.head(3)

,School,School ID,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,...,Student Survey College & Career Score,Student Survey College & Career Pts Earn,Student Survey College & Career Pts Poss,Student Survey College & Career Pct Earn,Student Survey College & Career Tier,Teach Effect Distinguished Score,Teach Effect Instruction Score,Teacher Attendance Score,Student Survey Teaching Score,Full Address
0,John Bartram High School,101,HS,High School,District,NaN,601,philasd.org/bartram/,'9-12',Neighborhood,...,19,0.19,1,19,INTERVENE,15,21,56,47,"2401 S 67th St, Philadelphia, PA 19142"
1,West Philadelphia High School,102,HS,High School,District,Turnaround,483,webgui.phila.k12.pa.us/schools/w/westphila,'9-12',Neighborhood,...,27,0.27,1,27,WATCH,11,29,56,42,"4901 Chestnut St, Philadelphia, PA 19139"
2,High School of the Future,103,HS,High School,District,NaN,476,webgui.phila.k12.pa.us/schools/h/high-school-o...,'9-12',Citywide,...,0,0,1,0,INTERVENE,16,13,50,0,"4021 Parkside Ave, Philadelphia, PA 19104"


# Get GPS Coordinates
For each school, use the address to find the Latitude and Longitude Coordinates. This will let us place the schools on a map.

In [13]:
geolocator = Nominatim()
cell_start = time.time()

lat = pd.Series({})
lon = pd.Series({})

for index, row in spr.iterrows():
    addr = row['Full Address']

    if row['School ID'] not in lat:
        # try geocoder 3 times
        for i in range(0,3):
            try:
                location = geolocator.geocode(addr)
                break
            except GeocoderTimedOut:
                print("Geocoder Timed Out. Address =", addr)
                continue

        try:
            latitude = location.latitude
            longitude = location.longitude
        except AttributeError:
            latitude = np.nan
            longitude = np.nan

        lat = pd.concat([lat, pd.Series({row['School ID']: latitude})])
        lon = pd.concat([lon, pd.Series({row['School ID']: longitude})])
    
    else:
        continue
        
print('Cell Runtime:', round(time.time() - cell_start), 'seconds')

Geocoder Timed Out. Address = 1928 Point Breeze Ave, Philadelphia, PA 19145
Geocoder Timed Out. Address = 1928 Point Breeze Ave, Philadelphia, PA 19145
Geocoder Timed Out. Address = 1928 Point Breeze Ave, Philadelphia, PA 19145
Cell Runtime: 127 seconds


In [14]:
print(lat.shape, sum(pd.isnull(lat)))
lat = lat.rename("Latitude")
lon = lon.rename("Longitude")

(294,) 44


In [15]:
temp1 = pd.merge(df2, spr, how='right', on='School ID')
temp2 = pd.merge(temp1, lat.to_frame(), how='left', left_on='School ID', right_index=True)
df3 = pd.merge(temp2, lon.to_frame(), how='left', left_on='School ID', right_index=True)
df3.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %,School,...,Student Survey College & Career Pts Poss,Student Survey College & Career Pct Earn,Student Survey College & Career Tier,Teach Effect Distinguished Score,Teach Effect Instruction Score,Teacher Attendance Score,Student Survey Teaching Score,Full Address,Latitude,Longitude
0,101,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476,93.5,John Bartram High School,...,1,19,INTERVENE,15,21,56,47,"2401 S 67th St, Philadelphia, PA 19142",39.921470,-75.234018
1,102,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415,94.0,West Philadelphia High School,...,1,27,WATCH,11,29,56,42,"4901 Chestnut St, Philadelphia, PA 19139",39.958587,-75.219431
2,103,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038,91.4,High School of the Future,...,1,0,INTERVENE,16,13,50,0,"4021 Parkside Ave, Philadelphia, PA 19104",39.974678,-75.204826
3,105,0.911765,0.970149,0.945205,0.960526,284.0,269.0,0.947183,97.0,Paul Robeson High School for Human Services,...,1,27,WATCH,15,15,61,55,"4125 Ludlow St, Philadelphia, PA 19104",39.956797,-75.204833
4,110,0.570370,0.475000,0.529801,0.530201,595.0,312.0,0.524370,92.6,William L. Sayre High School,...,1,24,INTERVENE,6,10,53,46,"5800 Walnut St, Philadelphia, PA 19139",39.958371,-75.237623


## Demographics Data

In [16]:
# Excel file has 4 worksheets, one fo each school type (Elem, K-8, MS, HS)
demog_dict = pd.read_excel('data/SPR_SY1617_Peer_Group_Data/SPR_SY1617_Peer_Group_Demographics_20180122.xlsx', \
                             sheet_name=[1,2,3,4], usecols="B:H,J:M")
# create common column naming
demog_dict[1] = demog_dict[1].rename(columns={'ES Group 1 (18 schools)': 'School ID'})
demog_dict[2] = demog_dict[2].rename(columns={'K8 Group 1 (26 schools)': 'School ID'})
demog_dict[3] = demog_dict[3].rename(columns={'MS Group 1 (10 schools)': 'School ID'})
demog_dict[4] = demog_dict[4].rename(columns={'HS Group 1 (22 schools)': 'School ID'})

# mark school type
demog_dict[1]['Report'] = 'ES'
demog_dict[2]['Report'] = 'K8'
demog_dict[3]['Report'] = 'MS'
demog_dict[4]['Report'] = 'HS'

# combine/concat
demog = pd.concat([demog_dict[1], demog_dict[2], demog_dict[3], demog_dict[4]])

# remove rows with blank school names
# because of the way the Excel file is laid out, these are non-data rows
demog = demog[~pd.isnull(demog['Unnamed: 1'])]

# drop School Name since we have it in the other DF
demog.drop('Unnamed: 1', axis=1, inplace=True)

demog.head()

,School ID,Low-Income,Special Education,English Learner,Black/ African American,Hispanic/ Latino,White,Asian/ Pacific Islander,American Indian/ Alaska Native,Other/ Multiple Races,Report
0,526,0.893,0.114,0.229,0.204,0.692,0.024,-,-,0.08,ES
1,553,0.879,0.125,0.142,0.253,0.626,0.027,0.007,0.004,0.082,ES
2,559,0.862,0.174,0.082,0.287,0.496,0.086,0.008,0.002,0.12,ES
3,547,0.862,0.148,0.278,0.14,0.759,0.017,0.004,-,0.08,ES
4,544,0.826,0.136,0.129,0.289,0.523,0.081,0.001,0.003,0.103,ES


In [17]:
# pd.set_option('display.max_columns', 500)
# df3.head()

In [18]:
df4 = pd.merge(df3, demog, how='outer', on=['School ID', 'Report'])
df4.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %,School,...,Longitude,Low-Income,Special Education,English Learner,Black/ African American,Hispanic/ Latino,White,Asian/ Pacific Islander,American Indian/ Alaska Native,Other/ Multiple Races
0,101,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476,93.5,John Bartram High School,...,-75.234018,0.718,0.291,0.083,0.915,0.03,0.007,0.03,0.002,0.017
1,102,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415,94.0,West Philadelphia High School,...,-75.219431,0.781,0.313,0.012,0.965,0.014,0.002,0.008,0.004,0.006
2,103,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038,91.4,High School of the Future,...,-75.204826,0.668,0.2,0.015,0.899,0.057,0.025,0.008,0.002,0.008
3,105,0.911765,0.970149,0.945205,0.960526,284.0,269.0,0.947183,97.0,Paul Robeson High School for Human Services,...,-75.204833,0.698,0.158,0.057,0.936,0.013,0.013,0.01,-,0.027
4,110,0.570370,0.475000,0.529801,0.530201,595.0,312.0,0.524370,92.6,William L. Sayre High School,...,-75.237623,0.773,0.323,0.004,0.957,0.006,0.014,0.002,0.002,0.018


# Export Data

In [19]:
df4.to_csv("test_data.csv", sep='|', index=True)

In [20]:
# print(min(lat), max(lat))
# print(min(lon), max(lon))